# CASO PRÁCTICO: GESTIÓN DE Atenciones

## 1. DETALLES GENERALES

- Cree una carpeta llamada 'Gestion Atenciones', en esta carpeta usted creará un entorno virtual con Python 3.11.7. Toda la implementación del caso práctico lo realizará conectado a ese entorno virtual.
- Descargue del classroom el Archivo Materiales.zip y descomprímalo dentro de la carpeta 'Gestion Atenciones'.
- En el entorno virtual instale la última versión de polars y el complemento xlsx2csv.

**Nota**: xIsx2csv, permite configurar los parámetros de conexión a un libro de Excel como si fuera un Archivo de texto plano.

## 2. TRANSFORMACIONES A REALIZAR

A continuación, encontrará el detalle de las transformaciones a realizar sobre cada una de las bases:

### BASE TICKETS

In [1]:
import polars as pl

- Importe la base 'Tickets Historico.txt', indicando que solo se importen las columnas: Numero Ticket, Ubicacion, Service Desk, Estado, Fecha Creacion, Fecha Termino y Fecha Cierre. Las columnas Fecha Creacion, Fecha Termino y Fecha Cierre deberán tener el tipo de dato fecha.
- Renombre la columna 'Numero Ticket' por 'TicketID'.
- Asigne el tipo de dato fecha a las columnas Fecha Creacion, Fecha Termino y Fecha Cierre.

In [2]:
# Definir las columnas a importar:
Columnas = ['Numero Ticket', 'Ubicacion', 'Service Desk', 'Estado', 'Fecha Creacion', 'Fecha Termino', 'Fecha Cierre']

# Cargar los Archivos, especificando el delimitador correcto y seleccionando las columnas especificadas:
Historico = pl.read_csv(r'Materiales/Tickets/Tickets Historico.txt', separator = ';', columns = Columnas) \
    .with_columns([
        # Asegurar que las fechas tengan el tipo de dato 'date':
        # Convertir 'Fecha Creacion', 'Fecha Termino' y 'Fecha Cierre' a fecha con el formato '%Y-%m-%d':
        pl.col('Fecha Creacion').str.strptime(pl.Date, '%Y-%m-%d'),
        pl.col('Fecha Termino').str.strptime(pl.Date, '%Y-%m-%d'),
        pl.col('Fecha Cierre').str.strptime(pl.Date, '%Y-%m-%d')
    ]) \
    .rename({
        # Renombrar la columna 'Numero Ticket' por 'TicketID':
        'Numero Ticket': 'TicketID'
    }) 

- Importe la base 'Tickets Actual.txt', indicando que solo se importen las columnas: Numero Ticket, Ubicacion, Service Desk, Estado, Fecha Creacion, Fecha Termino y Fecha Cierre. Las columnas Fecha Creacion, Fecha Termino y Fecha Cierre deberán tener el tipo de dato fecha.
- Renombre la columna 'Numero Ticket' por 'TicketID'.
- Asigne el tipo de dato fecha a las columnas Fecha Creacion, Fecha Termino y Fecha Cierre.
- Filtre la base actual de tal manera que solo se mantengan aquellos registros donde el TicketID inicia con WO.

In [3]:
# Definir las columnas a importar:
Columnas = ['Numero Ticket', 'Ubicacion', 'Service Desk', 'Estado', 'Fecha Creacion', 'Fecha Termino', 'Fecha Cierre']

# Cargar los Archivos, especificando el delimitador correcto y seleccionando las columnas especificadas:
Actual = pl.read_csv(r'Materiales/Tickets/Tickets Actual.csv', separator = '|', columns = Columnas) \
    .with_columns([
        # Asegurar que las fechas tengan el tipo de dato 'date':
        # Convertir 'Fecha Creacion' a fecha con el formato '%Y-%m-%d':
        pl.col('Fecha Creacion').str.strptime(pl.Date, '%Y-%m-%d'),
        # Convertir 'Fecha Termino' y 'Fecha Cierre' a fecha con el formato '%d/%m/%Y':
        pl.col('Fecha Termino').str.strptime(pl.Date, '%d/%m/%Y'),
        pl.col('Fecha Cierre').str.strptime(pl.Date, '%d/%m/%Y')
    ]) \
    .rename({
        # Renombrar la columna 'Numero Ticket' por 'TicketID':
        'Numero Ticket': 'TicketID'
    }) \
    .filter(
        # Filtrar para mantener registros donde el TicketID inicia con 'WO':
        pl.col('TicketID').str.starts_with('WO')
    )

- Anexe o concatene la base histórica y la base actual con el fin de crear un dataframe único llamado Tickets.
- En la base Tickets no deberían existir duplicados, usted deberá eliminar los duplicados basados en la base.
- Elimine los duplicados de la base Tickets en base a la siguiente regla: Si existe dos registros cuyo TicketID es igual, debe mantenerse aquel registro donde la [Fecha Creacion] sea la más actual.

In [4]:
# Concatenar los dataframes histórico y actual
Tickets = pl.concat([Historico, Actual]) \
    .sort(
        # Se ordena por 'Fecha Creacion' de forma descendente para tener el más reciente primero.
        ['TicketID', 'Fecha Creacion'], descending = [False, True]
    ) \
    .unique(
        # Eliminar duplicados manteniendo el primero de cada 'TicketID' y el registro con la [Fecha Creacion] más reciente.
        subset = ['TicketID'], keep = 'first', maintain_order = True
    )

- Dividir la columna [Ubicación], en las columnas [Agencia] y [AgencialD], usando como delimitador ' - '.
- Asignar el tipo de dato entero a la columna AgencialD.

In [5]:
Tickets = Tickets.with_columns([
        # Dividir la columna [Ubicación]:
        pl.col('Ubicacion')
        # Usando como delimitador ' - ':
        .str.split_exact(' - ', 1)
        # En las columnas [Agencia] y [AgencialD]:
        .struct.rename_fields( ['Agencia', 'AgenciaID'])
        .alias('Agencia')
    ]) \
    .unnest(
        'Agencia'
    ) \
    .cast(
        # Asignar el tipo de dato entero a la columna AgencialD:
        {'AgenciaID': pl.Int64}
    )

- Crear la columna [Fecha Real Fin] basado en la siguiente regla:
    - SI [Fecha Termino] es nulo ENTONCES [Fecha Cierre] SINO [Fecha Termino]

In [6]:
# Crear la columna [Fecha Real Fin]:
Tickets = Tickets.with_columns([
        # SI [Fecha Termino] es nulo:
        pl.when(pl.col('Fecha Termino').is_null())
        # ENTONCES [Fecha Cierre]:
        .then(pl.col('Fecha Cierre'))
        # SINO [Fecha Termino]:
        .otherwise(pl.col('Fecha Termino'))
        .alias('Fecha Real Fin')
    ])

- Crear la columna [Dias Cierre], la cual es la diferencia en días entre la [Fecha Real Fin] y [Fecha Creacion].

In [7]:
# Crear la columna [Dias Cierre]:
Tickets = Tickets.with_columns([
        # Diferencia en días entre [Fecha Real Fin] y [Fecha Creacion]
        (pl.col('Fecha Real Fin') - pl.col('Fecha Creacion')).dt.total_days().alias('Dias Cierre')
    ])

- Crear la columna [Grupo Dias] basado en la siguiente regla:
    - SI [Dias Cierre] es nulo ENTONCES Nulo
    - SI [Dias Cierre] <= 3 ENTONCES '0 a 3 días'
    - SI [Dias Cierre] <= 7 ENTONCES ' 4 a 7 días'
    - SI [Dias Cierre] <= 15 ENTONCES '8 a 15 días'
    - SI [Dias Cierre] > 15 ENTONCES '+15 días'

In [8]:
# Crear la columna [Grupo Dias]:
Tickets = Tickets.with_columns([
        # SI [Dias Cierre] es nulo ENTONCES Nulo:
        pl.when(pl.col('Dias Cierre').is_null()).then(None)
        # SI [Dias Cierre] <= 3 ENTONCES '0 a 3 días':
        .when(pl.col('Dias Cierre') <= 3).then(pl.lit('0 a 3 días'))
        # SI [Dias Cierre] <= 7 ENTONCES ' 4 a 7 días'
        .when(pl.col('Dias Cierre') <= 7).then(pl.lit('4 a 7 días'))
        # SI [Dias Cierre] <= 15 ENTONCES '8 a 15 días'
        .when(pl.col('Dias Cierre') <= 15).then(pl.lit('8 a 15 días'))
        # SI [Dias Cierre] > 15 ENTONCES '+15 días'
        .otherwise(pl.lit('+15 días'))
        .alias('Grupo Dias')
    ])

### BASE Atenciones

In [9]:
import os
import xlsxwriter

- Importe los Excel de la carpeta Atenciones, recuerde que la importación se debe realizar de manear masiva. De cada Excel solo se deberá importar las columnas: Numero Ticket, Tipo de Ticket, Proveedor y Costo Atencion; la columna Costo Atencion debe ser de tipo texto.
- La consulta anterior deberá almacenar el resultado en un dataframe llamado Atenciones.

In [10]:
# Definir la ruta de la carpeta que contiene los Archivos Excel:
RutaCarpeta = r'Materiales/Atenciones'
# Obtener la lista de Archivos Excel en la carpeta:
ArchivosExcel = [Archivo for Archivo in os.listdir(RutaCarpeta) if Archivo.endswith('.xlsx')]
# Inicializar un DataFrame vacío:
Atenciones = None
# Leer los Archivos Excel y concatenarlos en un solo DataFrame
for Archivo in ArchivosExcel:
    DF = pl.read_excel(os.path.join(RutaCarpeta, Archivo), 
                       read_options = {'columns': ['Numero Ticket', 'Tipo de Ticket', 'Proveedor', 'Costo Atencion'], 
                                       'dtypes': {'Costo Atencion': pl.Utf8}})
    # Seleccionar las columnas necesarias
    Atenciones = DF if Atenciones is None else Atenciones.vstack(DF)

- Cambiar el nombre de la columna 'Numero Ticket' por 'TicketID”.

In [11]:
Atenciones = Atenciones.rename({
        # Cambiar el nombre de la columna 'Numero Ticket' por 'TicketID'
        'Numero Ticket': 'TicketID'
    }) 

- Colocar en mayúscula los valores de la columna [Costo Atencion], luego realice el reemplace de la coma por el punto; así también, reemplace los textos 'SIN COSTO' y 'COSTO CERO' por el valor ' 0 '.
- Convertir la columna [Costo Atencion] al tipo de dato decimal, todos aquellos valores que no se puedan convertir deberían ser reemplazados por nulo.

In [12]:
Atenciones = Atenciones.with_columns([
        # Convertir la columna 'Costo Atencion' a mayúsculas
        pl.col('Costo Atencion').map_batches(lambda x: x.str.to_uppercase())
        .alias('Costo Atencion')
    ]) \
    .with_columns([
        # Reemplazar la coma por el punto en la columna 'Costo Atencion':
        pl.col('Costo Atencion').map_batches(lambda x: x.str.replace(',', '.'))
        .alias('Costo Atencion')
    ]) \
    .with_columns([
        # Reemplazar los textos 'SIN COSTO' y 'COSTO CERO' por el valor '0':
        pl.when(pl.col('Costo Atencion') == 'SIN COSTO').then(pl.lit('0'))
          .when(pl.col('Costo Atencion') == 'COSTO CERO').then(pl.lit('0'))
          .otherwise(pl.col('Costo Atencion'))
          .alias('Costo Atencion')
    ]) \
    .with_columns([
        # Convertir la columna 'Costo Atencion' al tipo de dato decimal (Float64):
        pl.col('Costo Atencion').cast(pl.Float64, strict = False)
        .alias('Costo Atencion')
    ])

### COMBINAR Y EXPORTAR

- Realice una combinación de tipo Inner Join entre los dataframe Tickets y Atenciones, usando como columna del match [TicketID].
- Del punto anterior usted deberá extraer solo las columnas: TicketID, AgencialD, Agencia, Service Desk, Estado, Fecha Creacion, Fecha Real Fin, Grupo Dias, Tipo de Ticket y Costo Atencion.
- Cambie el nombre de las columnas [Fecha Real Fin] por [Fecha Cierre], [Tipo de Ticket] por [Tipo Ticket] y [Costo Atencion] por [Costo].

In [13]:
# Definir las columnas requeridas:
Columnas = [
    'TicketID',       'AgenciaID',      'Agencia',    'Service Desk',   'Estado', 
    'Fecha Creacion', 'Fecha Real Fin', 'Grupo Dias', 'Tipo de Ticket', 'Costo Atencion'
]
# Definir los nuevos nombres de columna:
NuevosNombres = {
    'Fecha Real Fin': 'Fecha Cierre',
    'Tipo de Ticket': 'Tipo Ticket',
    'Costo Atencion': 'Costo'
}
# Realizar la combinación de tipo Inner Join:
Consolidado = Tickets.join(Atenciones, on = 'TicketID', how = 'inner') \
    .select(
        # Seleccionar las columnas específicas:
        Columnas
    ) \
    .rename(
        # Cambiar el nombre de las columnas:
        NuevosNombres
    ) \
    .with_columns([
        # Formatear las fechas de la columna 'Fecha Creacion' al formato 'dd/mm/yyyy':
        pl.col('Fecha Creacion').cast(pl.Date).dt.strftime('%d/%m/%Y').alias('Fecha Creacion')
    ]) \
    .with_columns([
        # Formatear las fechas de la columna 'Fecha Cierre' al formato 'dd/mm/yyyy':
        pl.col('Fecha Cierre').cast(pl.Date).dt.strftime('%d/%m/%Y').alias('Fecha Cierre')
    ]) \
    .with_columns([
        # Formatear los números decimales para tener solo 2 dígitos en la parte decimal:
        pl.col('Costo').round(2).alias('Costo')
    ])

- Exporte la base consolidada en una hoja llamada 'Atenciones' perteneciente a un libro de excel llamado 'Consolidado.xIsx'.

**Nota**: Recuerde que el formato de la fecha debe ser de tipo 'dd/mm/yyyy', además para los números decimales se debe tener solo 2 digitos en la parte decimal.

In [14]:
Consolidado.write_excel(
        # Exportar la base consolidada a un archivo Excel:
        workbook = 'Consolidado.xlsx', worksheet ='Atenciones', column_totals = True, autofit = True
    )

In [15]:
Consolidado

TicketID,AgenciaID,Agencia,Service Desk,Estado,Fecha Creacion,Fecha Cierre,Grupo Dias,Tipo Ticket,Costo
str,i64,str,str,str,str,str,str,str,f64
"""WO000000558468…",191076,"""LAS MALVINAS""","""Zona Centro""","""Cerrado""","""08/09/2023""","""17/09/2023""","""8 a 15 días""","""Variable""",null
"""WO000000454494…",191053,"""URB SAN FELIPE…","""Zona Centro""","""Rechazado""","""18/10/2022""","""18/10/2022""","""0 a 3 días""","""Rechazado""",100.0
"""WO000000454524…",191035,"""PUENTE PIEDRA""","""Zona Centro""","""Rechazado""","""18/10/2022""","""18/10/2022""","""0 a 3 días""","""Rechazado""",100.0
"""WO000000455912…",191087,"""EL PINO""","""Zona Centro""","""Rechazado""","""24/10/2022""","""25/10/2022""","""0 a 3 días""","""Rechazado""",100.0
"""WO000000470747…",191044,"""LAS ALONDRAS""","""Zona Centro""","""Rechazado""","""08/11/2022""",null,null,"""Rechazado""",100.0
"""WO000000475997…",193022,"""ROSA TORO""","""Zona Centro""","""Rechazado""","""26/11/2022""",null,null,"""Rechazado""",100.0
"""WO000000479587…",470000,"""PISCO""","""Zona Centro""","""Rechazado""","""09/12/2022""",null,null,"""Rechazado""",100.0
"""WO000000482530…",191058,"""PLAZA LIMA NOR…","""Zona Centro""","""Rechazado""","""19/12/2022""",null,null,"""Rechazado""",100.0
"""WO000000485368…",192026,"""PLAZA CIVICA V…","""Zona Centro""","""Cerrado""","""02/01/2023""","""08/01/2023""","""4 a 7 días""","""Flat""",100.0
